## KNN from scratch

###### By Yasmine TOLBA

In [320]:
#imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Import and split the dataset

In [321]:
import csv
#check out the data file with a txt format
with open('iris.data.txt', 'r') as csvfile:
  lines = csv.reader(csvfile)
  for row in lines :
    print(', '.join(row))


5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4

In [322]:

import csv

import random
#split the dataset randomly
def loadDataset(filename, split, trainingSet=[] , testSet=[]):

  with open(filename, 'r') as csvfile:

    lines = csv.reader(csvfile)

    dataset = list(lines)

    for x in range(len(dataset)-1):

      for y in range(4):

        dataset[x][y] = float(dataset[x][y])

      if random.random() < split:
        trainingSet.append(dataset[x])

      else:
        testSet.append(dataset[x])


training_set=[]

test_set=[]

loadDataset('iris.data.txt', 0.66, training_set, test_set)

print('Train: ' + repr(len(training_set)))

print('Test: ' + repr(len(test_set)) )

Train: 100
Test: 49


3 different functions, for 3 different distances.<br>
we could remove 2 and only leave the minkowski distance and add a p parameter instead of a hard coded value, and that would give us both the euclidean distance when p=2, and the manhattan distance when p=1.
but for the sake of this checkpoint, we made the minkowski distance just like it was portrayed in the gomycode course, so it's right in between euclidean and manhattan, with a p=3/2.

In [323]:
import math

def euclideanDistance(instance1, instance2, length):
        return math.sqrt(sum(list(map(lambda x,y: math.pow(x-y,2),instance1[:length],instance2[:length]))))


In [324]:
def manhattanDistance(instance1, instance2, length):
        return sum(list(map(lambda x, y: abs(x-y), instance1[:length], instance2[:length])))

In [325]:
def minkowskiDistance(instance1, instance2, length):
        return math.pow(sum(list(map(lambda x, y: math.pow(abs(x-y),(3/2)), instance1[:length], instance2[:length]))), 2/3)

In [326]:
data1 = [2, 2, 2, 'a']

data2 = [4, 4, 4, 'b']

distance = euclideanDistance(data1, data2, 3)
print ('Euclidean Distance: ' + repr(distance))

distance = manhattanDistance(data1, data2, 3)
print ('Manhattan Distance: ' + repr(distance))

distance = minkowskiDistance(data1, data2, 3)
print ('Minkowski Distance: ' + repr(distance))

Euclidean Distance: 3.4641016151377544
Manhattan Distance: 6
Minkowski Distance: 4.160167646103808


In [327]:
import operator
#returns the k nearest neighbors
def getNeighbors(trainingSet, testInstance, k, distance = 'Euclidean'):

        distances = []

        length = len(testInstance)-1
        # calculting the distance between the instance and each instance in the training dataset
        for x in range(len(trainingSet)):
                #choosing which distance to use, by default it's euclidean
                if(distance == "Euclidean"):
                        dist = euclideanDistance(testInstance, trainingSet[x], length)
                elif(distance == "Manhattan"):
                        dist = manhattanDistance(testInstance, trainingSet[x], length)
                elif(distance == "Minkowski"):
                        dist = minkowskiDistance(testInstance, trainingSet[x], length)
                else: print("an error has occured")

                distances.append((trainingSet[x], dist))
        # we sort the distances
        distances.sort(key=operator.itemgetter(1))

        neighbors = []

        for x in range(k):
                #we take the k nearest neighbors, so the instances with the shortest distance
                neighbors.append(distances[x][0])

        return neighbors



trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]

testInstance = [5, 5, 5]

k = 1

neighbors = getNeighbors(trainSet, testInstance, k)

print(neighbors)

[[4, 4, 4, 'b']]


In [328]:
import operator

def getResponse(neighbors):
        #this function does the majority vote by using the k nearest neighbors, to classify our instance
        classVotes = {}

        for x in range(len(neighbors)):

                response = neighbors[x][-1] #complete with appropriate number
                
                if response in classVotes:
                        classVotes[response] += 1
                else: classVotes[response] = 1
        #it's a dictionary with the class as a key and the occurence of that class within the knn
        sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)

        return sortedVotes[0][0]


neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]

response = getResponse(neighbors)

print(response)

a


In [329]:
def getAccuracy(testSet, predictions):
        correct = 0
        for x in range(len(testSet)):
                if testSet[x][-1] == predictions[x]: correct +=1
        return (correct/float(len(testSet))) * 100.0


testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]

predictions = ['a', 'a', 'a']

accuracy = getAccuracy(testSet, predictions)

print(accuracy)

66.66666666666666


Full KNN

In [330]:

def KNN(dataset, instance, k=3, distance = 'Euclidean'):
        return getResponse(getNeighbors(dataset,instance, k, distance))


In [331]:
#since the dataset split is random, the values can vary, and they can even all get the same accuracy
print(getAccuracy(test_set, list(map(lambda x: KNN(training_set, x, k=7), test_set))))
print(getAccuracy(test_set, list(map(lambda x: KNN(training_set, x, distance="Manhattan", k=7), test_set))))
print(getAccuracy(test_set, list(map(lambda x: KNN(training_set, x, distance="Minkowski", k=7), test_set))))
#i set the k as 7 in order to have a wider range of accuracies, as a k 3 and 5 basically gave the same accuracy

97.95918367346938
89.79591836734694
95.91836734693877
